<a href="https://colab.research.google.com/github/bennykan/ML1020/blob/master/Ml1020_Pretrained_Model_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
sys.version
import keras

Using TensorFlow backend.


In [3]:
!pwd

/home/jupyter/ML1020/ML1020


In [4]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = '/home/jupyter/ML1020/ML1020'

train_dir = os.path.join(base_dir, 'Sample')
print(train_dir)
#validation_dir = os.path.join(base_dir, 'validation')
#test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 10

train_features = np.zeros(shape=(200, 15, 20, 512))
train_labels = np.zeros(shape=(200,10))
 
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(480, 640),
    batch_size=batch_size,
    class_mode='categorical')

/home/jupyter/ML1020/ML1020/Sample
Found 200 images belonging to 10 classes.


In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread(base_dir + '/train/c6/img_380.jpg')
img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_cvt)
plt.show()

error: OpenCV(4.0.0) /io/opencv/modules/imgproc/src/color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [6]:
#Loading in Pretrained Model

from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(480, 640, 3))

In [7]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 480, 640, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 480, 640, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 480, 640, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 240, 320, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 240, 320, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 240, 320, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 120, 160, 128)     0         
__________

In [8]:
#Feature engineering

i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = conv_base.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= 200:
        break
print(train_features.shape)         
train_features = np.reshape(train_features, (200, 15 * 20 * 512))

(200, 15, 20, 512)


In [11]:
print(train_labels)
print(train_features)
print(train_labels.shape)
print(train_features.shape)




[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0.01537259 0.         0.         ... 0.         0.96648932 0.        ]
 [0.         0.         0.         ... 0.         0.94795364 0.        ]
 [0.         0.         0.         ... 0.         0.89855254 0.        ]
 ...
 [0.0930624  0.         0.         ... 0.         0.77023864 0.        ]
 [0.         0.         0.         ... 0.         0.6494078  0.        ]
 [0.02176097 0.         0.         ... 0.         0.74609101 0.        ]]
(200, 10)
(200, 153600)


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2)

In [10]:
print(X_train)
print(X_train.shape)
print(X_test.shape)

[[0.         0.         0.         ... 0.         1.37048173 0.        ]
 [0.         0.         0.         ... 0.         0.83549976 0.        ]
 [0.         0.         0.         ... 0.         1.01423395 0.        ]
 ...
 [0.         0.         0.         ... 0.11735116 0.75239027 0.        ]
 [0.         0.         0.07238156 ... 0.         0.64992285 0.        ]
 [0.10070539 0.         0.         ... 0.01925092 0.94601667 0.        ]]
(160, 153600)
(40, 153600)


In [11]:
# Create NN to classifer 10 outputs
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()


model.add(layers.Dense(256, activation='relu', input_dim=15 * 20 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation = 'softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_features, train_labels,
                    epochs=7,
                    batch_size=20,
                   validation_data=(X_test,y_test))




model.summary()

Train on 200 samples, validate on 40 samples
Epoch 1/7
200/200 [==============================] - 8s 40ms/step - loss: 3.1790 - acc: 0.1100 - val_loss: 2.0182 - val_acc: 0.2000
Epoch 2/7
200/200 [==============================] - 1s 3ms/step - loss: 2.1415 - acc: 0.2250 - val_loss: 1.7605 - val_acc: 0.6000
Epoch 3/7
200/200 [==============================] - 1s 3ms/step - loss: 1.8888 - acc: 0.3700 - val_loss: 1.4971 - val_acc: 0.5750
Epoch 4/7
200/200 [==============================] - 1s 3ms/step - loss: 1.7368 - acc: 0.4000 - val_loss: 1.2335 - val_acc: 0.8000
Epoch 5/7
200/200 [==============================] - 1s 3ms/step - loss: 1.5478 - acc: 0.4750 - val_loss: 1.0344 - val_acc: 0.8250
Epoch 6/7
200/200 [==============================] - 1s 3ms/step - loss: 1.3340 - acc: 0.5700 - val_loss: 0.8746 - val_acc: 0.9250
Epoch 7/7
200/200 [==============================] - 1s 3ms/step - loss: 1.1440 - acc: 0.6950 - val_loss: 0.7399 - val_acc: 0.9250
_____________________________________

Below is the list of testing labels and their descriptions



*   c0: safe driving
*   c1: texting - right
*   c2: talking on the phone - right
*   c3: texting - left
*   c4: talking on the phone - left
*   c5: operating the radio
*   c6: drinking
*   c7: reaching behind
*   c8: hair and makeup
*   c9: talking to passenger


In [12]:
ls "/home/jupyter/TEST data"

img_101.jpg  img_25136.jpg  img_41068.jpg  img_71188.jpg  img_97377.jpg
img_11.jpg   img_31.jpg     img_49714.jpg  img_71233.jpg  img_97383.jpg
img_165.jpg  img_326.jpg    img_58795.jpg  img_80278.jpg  img_97390.jpg
img_227.jpg  img_39.jpg     img_71146.jpg  img_80282.jpg  img_97396.jpg


In [13]:
import pandas as pd
import glob

output_dir = '/home/jupyter/'
test_base_dir = '/home/jupyter/TEST data/'
prediction_list = []

for file in glob.glob(test_base_dir + '*jpg'):
  dirname, filename = os.path.split(file)
  pred_row = [filename]
  img = cv2.imread(file)
  img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  testimage = conv_base.predict(img_cvt.reshape(1,480,640,3))
  testimage_rs = np.reshape(testimage, (1, 15 * 20 * 512))
  pred = model.predict_proba(testimage_rs)
  pred_list = pred[0].tolist()
  pred_row = pred_row + pred_list
  prediction_list.append(pred_row)

df_results = pd.DataFrame(prediction_list,columns=['img','c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'])
df_results.to_csv(output_dir + 'sample_submission.csv',index=False)
print(df_results)





              img            c0            c1            c2            c3  \
0      img_31.jpg  6.669070e-29  2.709959e-10  5.295300e-09  1.228660e-10   
1   img_80282.jpg  5.722410e-12  1.000000e+00  1.339381e-11  2.073463e-10   
2   img_97377.jpg  5.508116e-14  2.656782e-07  2.291770e-04  8.108423e-10   
3     img_227.jpg  3.153502e-26  9.310229e-18  5.727428e-17  4.122539e-04   
4     img_326.jpg  5.130187e-05  2.091885e-22  3.792549e-10  5.329054e-03   
5   img_97383.jpg  1.111705e-13  1.687699e-13  1.000000e+00  4.292856e-13   
6   img_41068.jpg  3.394101e-04  1.779419e-08  7.977716e-02  1.221625e-05   
7   img_71146.jpg  2.401002e-20  1.272794e-15  5.335541e-08  9.999999e-01   
8   img_80278.jpg  1.161966e-21  5.781484e-10  8.476240e-10  1.466823e-08   
9   img_25136.jpg  2.096712e-23  3.851835e-11  3.619238e-07  9.846943e-01   
10     img_39.jpg  2.775348e-19  2.248719e-05  2.930307e-14  2.928659e-03   
11  img_49714.jpg  6.980718e-08  9.365958e-01  5.764058e-02  5.734445e-04   